In [ ]:
import os
from pathlib import Path

DATA_ROOT = "/kaggle/input/data-final/Data"
DATA_YAML = os.path.join(DATA_ROOT, "data.yaml")
YOLO_ROOT = "/kaggle/working/Yolo_Experiments_2"  # ← AQUÍ DONDE SE GUARDAN LOS RESULTADOS

os.makedirs(YOLO_ROOT, exist_ok=True)

print("📁 Dataset montado en:", DATA_ROOT)
print("✅ data.yaml existe:", os.path.exists(DATA_YAML))
print("📂 YOLO_ROOT creado en:", YOLO_ROOT)

print("\n📂 Contenido dataset:")
for item in os.listdir(DATA_ROOT):
    item_path = os.path.join(DATA_ROOT, item)
    if os.path.isdir(item_path):
        img_count = len(os.listdir(os.path.join(item_path, "images")))
        print(f"  📁 {item}/ ({img_count} imágenes)")
    else:
        print(f"  📄 {item}")


📁 Dataset montado en: /kaggle/input/data-final/Data
✅ data.yaml existe: True
📂 YOLO_ROOT creado en: /kaggle/working/Yolo_Experiments_2

📂 Contenido dataset:
  📄 data.yaml
  📁 valid/ (150 imágenes)
  📁 test/ (134 imágenes)
  📁 train/ (7771 imágenes)


In [5]:
!pip install ultralytics optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0:00:0000:01


In [6]:
from ultralytics import YOLO
import torch
import os
import optuna
import json

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", DEVICE)


Device: cuda


In [12]:
DATA_YAML = "/kaggle/working/data.yaml" 

def objective(trial):
    # Hyperparametros para optuna
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])

    # Modelo
    model = YOLO("yolov8l.pt")

    model.train(
        data=DATA_YAML,
        epochs=10, 
        patience=3,  # para si no mejora mAP durante 3 épocas consecutivas
        imgsz=512,
        batch=-1,  # YOLOv8 calcula AUTOMÁTICAMENTE el máximo batch_size que cabe en tu GPU
        lr0=lr,
        optimizer=optimizer_name,
        freeze=10,      # congelar primeras capas (backbone)
        device=DEVICE,  # cuda
        project=YOLO_ROOT,
        name=f"trial_{trial.number}",
        save=True,      # guardar best.pt / last.pt
        exist_ok=True,  # no sobrescribir conflictos
        cache=False,
        verbose=True
    )

    # Evaluación final
    val_results = model.val()

    return 1 - float(val_results.box.map50)


In [13]:
# Crear estudio 
study = optuna.create_study(direction="minimize")

# Ejecutar Optuna
study.optimize(objective, n_trials=3)

print("Mejor configuración encontrada:")
print(study.best_params)


[I 2026-01-24 04:19:54,463] A new study created in memory with name: no-name-38db5917-6921-401e-b8b8-37369b69ae38


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.009311958515851737, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Yo

[I 2026-01-24 04:49:43,684] Trial 0 finished with value: 0.26701214559036124 and parameters: {'lr': 0.009311958515851737, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.26701214559036124.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025374738430599376, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Y

[I 2026-01-24 05:20:12,342] Trial 1 finished with value: 0.2283620695253017 and parameters: {'lr': 0.0025374738430599376, 'optimizer': 'AdamW'}. Best is trial 1 with value: 0.2283620695253017.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005133073345812616, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Yol

[I 2026-01-24 05:51:02,243] Trial 2 finished with value: 0.25103311092511693 and parameters: {'lr': 0.005133073345812616, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.2283620695253017.


Mejor configuración encontrada:
{'lr': 0.0025374738430599376, 'optimizer': 'AdamW'}


In [ ]:
print("🏆 RESULTADOS FINALES Optuna:")
print(f"Total trials: {len(study.trials)}")
print(f"Mejor trial: {study.best_trial.number}")
print(f"Mejor params: {study.best_params}")
print(f"Mejor mAP50: {1 - study.best_value:.3f}")

🏆 RESULTADOS FINALES Optuna:
Total trials: 3
Mejor trial: 1
Mejor params: {'lr': 0.0025374738430599376, 'optimizer': 'AdamW'}
Mejor mAP50: 0.772


In [ ]:
# Cargar el mejor trial
best_trial_number = study.best_trial.number
best_trial_dir = os.path.join(YOLO_ROOT, f"trial_{best_trial_number}")
best_model_path = os.path.join(best_trial_dir, "weights", "best.pt")

# Cargar modelo YOLO
from ultralytics import YOLO
best_model = YOLO(best_model_path)

# Evaluar validación final
results = best_model.val(data=DATA_YAML)

print(f"\n VALIDACIÓN:")
print(f"mAP50:     {results.box.map50:.3f}")
print(f"mAP50-95:  {results.box.map:.3f}")
print(f"Precision: {results.box.mp:.3f}")
print(f"Recall:    {results.box.mr:.3f}")

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 113 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 24.2±19.4 MB/s, size: 39.3 KB)
val: Scanning /kaggle/input/data-final/Data/valid/labels... 150 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 150/150 590.3it/s 0.3s0.1s
val: /kaggle/input/data-final/Data/valid/images/Fv1_0950.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0952.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0957.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0961.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0968.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0977.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0982.jpg: 3 duplicate lab

In [9]:
# Cargar MEJOR config encontrada
DATA_YAML = '/kaggle/working/data_final.yaml' 

best_params = {
    'lr': 0.0025374738430599376,           
    'optimizer': 'AdamW'    
}

model = YOLO("yolov8l.pt")  

results = model.train(
    data=DATA_YAML,
    epochs=20, 
    imgsz=512,
    batch=-1,
    lr0=best_params['lr'],        
    optimizer=best_params['optimizer'],
    freeze=10,
    device=DEVICE,
    patience=2,                   
    project=YOLO_ROOT,
    name="FINAL_MEJORADO",
    cache=False,
    verbose=True
)

engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_final.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025374738430599376, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=FINAL_MEJORADO, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=2, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/ka

In [11]:
model = YOLO(f"{YOLO_ROOT}/FINAL_MEJORADO/weights/best.pt")

print("VALIDACIÓN:")
val_results = model.val(data=DATA_YAML, split='val')
print(f"Val mAP50: {val_results.box.map50:.3f}")

VALIDACIÓN:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 113 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.3 ms, read: 121.8±49.7 MB/s, size: 68.0 KB)
val: Scanning /kaggle/input/data-final/Data/valid/labels... 150 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 150/150 753.9it/s 0.2s3s
val: /kaggle/input/data-final/Data/valid/images/Fv1_0950.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0952.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0957.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0961.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0968.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0977.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0982.jpg: 3 du

In [12]:
print("\nTEST FINAL:")
test_results = model.val(data='/kaggle/working/data_final.yaml', split='test')
print(f"**TEST mAP50:** {test_results.box.map50:.3f}")
print(f"Test mAP50-95: {test_results.box.map:.3f}")
print(f"Test Precision: {test_results.box.mp:.3f}")
print(f"Test Recall: {test_results.box.mr:.3f}")


TEST FINAL:
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 8.6±7.2 MB/s, size: 53.0 KB)
val: Scanning /kaggle/input/test-completo/test/labels... 147 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 147/147 261.2it/s 0.6s0.1s
val: /kaggle/input/test-completo/test/images/Fv1_1019.jpg: 3 duplicate labels removed
val: /kaggle/input/test-completo/test/images/Fv1_1037.jpg: 2 duplicate labels removed
val: /kaggle/input/test-completo/test/images/Fv1_1045.jpg: 1 duplicate labels removed
val: /kaggle/input/test-completo/test/images/Fv1_1052.jpg: 1 duplicate labels removed
WARNING ⚠️ val: Cache directory /kaggle/input/test-completo/test is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 1.8it/s 5.4s.5ss
                   all        147       1874      0.666      0.528      0.552      0.335
                  door  